# McNemar's

In [5]:
import csv
import os
import codecs

data_dirpath = '/usr2/mamille2/tumblr/data/sample1k'

feature_tables_dir = os.path.join(data_dirpath, 'feature_tables')
filenames = ['reblog_features.csv', 'nonreblog_features.csv', 'ranking_labels.csv']
joined_filenames = [os.path.join(feature_tables_dir, filename) for filename in filenames]
# csv_readers = [csv.DictReader(codecs.open(filename, 'rU', 'utf-16')) for filename in joined_filenames]
csv_readers = [csv.DictReader(x.replace('\0', '') for x in open(filename, 'r')) for filename in joined_filenames]

In [6]:
instances = []
instance_labels = []
for row in zip(*csv_readers):
    reblog_features = row[0]
    nonreblog_features = row[1]
    label = int(row[2]['ranking_label'])
    instance = (reblog_features, nonreblog_features) # reblog always first, nonreblog always second
    instances.append(instance)
    instance_labels.append(label)
    
print(len(instances), len(instance_labels))

712670 712670


In [9]:
from sklearn import model_selection

X = []
y = []
for (reblog_candidate, nonreblog_candidate), label in zip(instances, instance_labels):
    X.append([])
    y.append(label)
    
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2, random_state=12345)

In [3]:
import numpy as np
import os
data_dirpath = '/usr2/mamille2/tumblr/data/sample1k'

baseline_pred = np.loadtxt(os.path.join(data_dirpath, 'output', 'predictions', 'lr_baseline+exp1_all_test_preds.txt'))
experiment_pred = np.loadtxt(os.path.join(data_dirpath, 'output', 'predictions', 'lr_baseline+exp2_all_test_preds.txt'))
print(baseline_pred.shape)
print(experiment_pred.shape)

(71267,)
(71267,)


In [10]:
a = 0
b = 0 # Baseline correct, experiment incorrect
c = 0 # Baseline incorrect, experiment correct
d = 0
for b_pred, ex_pred, true in zip(baseline_pred, experiment_pred, y_test):
    if b_pred == true and ex_pred == true:
        a += 1
    elif b_pred == true and ex_pred != true:
        b += 1
    elif b_pred != true and ex_pred == true:
        c += 1
    else:
        d += 1
        
table = [[a, b],
         [c, d]]
print(table)

[[41740, 4450], [6404, 18673]]


In [11]:
# Example of calculating the mcnemar test
from statsmodels.stats.contingency_tables import mcnemar
# calculate mcnemar test
result = mcnemar(table, exact=False, correction=False)
# summarize the finding
print('statistic=%.3f, p-value=%.6f' % (result.statistic, result.pvalue))
# interpret the p-value
alpha = 0.05
if result.pvalue > alpha:
	print('Same proportions of errors (fail to reject H0)')
else:
	print('Different proportions of errors (reject H0)')

statistic=351.770, p-value=0.000000
Different proportions of errors (reject H0)
